# Einfaktorielle ANOVAs

Im letzten Notebook haben wir uns angesehen wie man in Python T-Tests ausführt. Das Problem am Ende des Notebooks war, dass wir drei Gruppen Pinguine haben, bei denen wir die Schnabellänge verglichen haben, was letztendlich zu vielen einzelnen Tests führte und damit zur sogenannten Alpha-Fehler Kummulierung. 

Stattdessen kann man auch in einem einzelnen Test eine ANOVA durchführen und so die Mittelwerte von mehr als zwei Gruppen vergleichen und so herauszufinden, ob sich die Schnabellänge zwischen den drei Pinguingruppen unterschiedet.

In [1]:
import pandas as pd

penguins = pd.read_csv(
    "./penguins_classification.csv"
)  # Der Punkt ist ein 'relativer Pfad', der dazu führt, dass im selben Ordner nach den Daten gesucht wird, wie das Skript ist.

Für die ANOVA nutzen wir wieder das Pingouin Modul. Man kann auch einzelne Befehle aus Modulen importieren. Aus dem Pingouin Modul importieren wir also den Befehl, der eine ANOVA berechnet.

In [2]:
from pingouin import anova

Die Dokumentation können wir mit ? vor dem Befehl, den wir ausführen wollen öffnen.

In [3]:
?anova

Signature:
anova(
    data=None,
    dv=None,
    between=None,
    ss_type=2,
    detailed=False,
    effsize='np2',
)
Docstring:
One-way and *N*-way ANOVA.

Parameters
----------
data : :py:class:`pandas.DataFrame`
    DataFrame. Note that this function can also directly be used as a
    Pandas method, in which case this argument is no longer needed.
dv : string
    Name of column in ``data`` containing the dependent variable.
between : string or list with *N* elements
    Name of column(s) in ``data`` containing the between-subject factor(s).
    If ``between`` is a single string, a one-way ANOVA is computed.
    If ``between`` is a list with two or more elements, a *N*-way ANOVA is
    performed.
    Note that Pingouin will internally call statsmodels to calculate
    ANOVA with 3 or more factors, or unbalanced two-way ANOVA.
ss_type : int
    Specify how the sums of squares is calculated for *unbalanced* design
    with 2 or more factors. Can be 1, 2 (default), or 3. This has no i

Dieselbe Dokumentation können wir <a href="https://pingouin-stats.org/generated/pingouin.anova.html#pingouin.anova">online</a> finden. Hierbei werden die mathematischen Formeln korrekt angezeigt.

Q: Welche Voraussetzungen sollten laut der Dokumentation für eine ANOVA erfüllt werden? 

Q: Prüfe (visuell) die Voraussetzungen für die ANOVA. 

In [4]:
anova(data=penguins, dv="Culmen Length (mm)", between="Species", detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,Species,7194.317439,2,3597.158720,410.600255,2.694614e-91,0.707809
1,Within,2969.888087,339,8.760732,NaN,NaN,NaN


Die Berechnung der Freiheitsgrade (engl. *degrees of freedom*, DF), der *Sum of Squares* (SS), *Mean Squares* (MS), F-Werte (F), p-Werte (p-unc) und Effektstärke (als partial eta [$\eta$] Quadrat) sind in der Dokumentation berichtet. Diese bietet uns auch Hilfe bei der Interpretation der Werte.

Der F-Wert, die Tatsache, dass es zwei Freiheitsgrade gibt und auch einige Beschriftungen der Tabelle lässt sich unter anderem damit erklären, dass die ANOVA den Mittelwertsunterschied testet, indem verschiedene Varianzen (Between und within der Gruppen) verglichen werden.

Der F-Wert berechnet sich zum Beispiel aus:

$$F = \frac{Signal}{Noise} = \frac{Explained\:variance}{Unexplained\:variance} = \frac{Between\:group\:variability}{Within\:group\:variability}$$

Wenn man eine ANOVA berichtet, werden die Freiheitsgrade, der F-Wert, der P-Wert und im besten Fall noch die Effektstärke berichtet. Für die Freiheitsgrade werden sowohl die des untersuchten (between) Faktors (2) als auch die des (within Faktors) Residuums (339), also der übrigen Varianz, berichtet.

Hierbei sollte eine gewisse Form mit kursiven Buchstaben und Leerzeichen beachtet werden. 

*F*(df1, df2) = F-Wert, *p* = .p-Wert (ohne 0 vor dem Komma, p-Wert unter .001 als *p* < .001) <br><br>
Hier also: <br><br>
*F*(2, 339) = 410.60, *p* < .001

Wie dieser Effekt zustande kommt lässt sich anhand der Mittelwerte ablesen. Mittelwerte und Standardabweichungen für die Gruppen sollten auch berichtet werden, da dies Meta-Analysen erleichtert. Einen kurzen Überblick können wir uns mit einem Boxplot verschaffen. Das Pandas Modul ermöglicht aus Dataframes sehr schnell Plots zu erstellen.

In [ ]:
penguins.boxplot(column="Culmen Length (mm)", by="Species", figsize=(12, 8));

Q: Welche Pinguingruppe treibt wahrscheinlich den Effekt an?

Dies lässt sich allerdings auch mit sogenannten Post-hoc Tests überprüfen. Die Pingouin Dokumentation kann hierbei auch helfen, den <a href="https://pingouin-stats.org/generated/pingouin.pairwise_tukey.html#pingouin.pairwise_tukey">richtigen</a> Post-hoc Test zu finden. 

Q: Kannst du allein mit der Dokumenation herausfinden, wie der Post-hoc Test in Python durchgeführt und interpretiert wird? 

In [ ]:
# Lösung

from pingouin import (
    pairwise_tukey as tukey,
)  # eine weitere Möglichkeit des Imports, bei der dem Befehl ein neuer Name zugeordnet wird.

tukey(
    data=penguins,
    dv="Culmen Length (mm)",
    between="Species",
)

"""
Wir sehen, dass der Mittelwert von allen drei Gruppen signifikant voneinander abweichen (p-tukey). 
Trotzdem zeigt uns die Effektstärke, dass besonders der Vergleich mit der Adelie Spezies zu großen Effekten führt (hedges).
"""